### Imports

In [1]:
import torch
print(f'PyTorch CUDA is available? {torch.cuda.is_available()}')
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

PyTorch CUDA is available? True


### Fix randomness => reproducibility

In [2]:
def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.

    Args:
        seed: the seed to use.
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(42)

## Devices availables

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Device: {device}")

Device: cuda


## Access dataloader

In [ ]:
!git clone https://github.com/marcusklasson/GroceryStoreDataset.git

In [4]:
from pathlib import Path
from PIL import Image
from torch import Tensor
from torch.utils.data import Dataset
from typing import List, Tuple

In [5]:
class GroceryStoreDataset(Dataset):

    def __init__(self, split: str, transform=None) -> None:
        super().__init__()

        self.root = Path("GroceryStoreDataset/dataset")
        self.split = split
        self.paths, self.labels = self.read_file()

        self.transform = transform

    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, idx) -> Tuple[Tensor, int]:
        img = Image.open(self.root / self.paths[idx])
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

    def read_file(self) -> Tuple[List[str], List[int]]:
        paths = []
        labels = []

        with open(self.root / f"{self.split}.txt") as f:
            for line in f:
                # path, fine-grained class, coarse-grained class
                path, _, label = line.replace("\n", "").split(", ")
                paths.append(path), labels.append(int(label))

        return paths, labels

    def get_num_classes(self) -> int:
        return max(self.labels) + 1

In [6]:
from torchvision import transforms as T, datasets

In [64]:
tsfms_std = T.Compose([
    T.Resize(size=(260, 260)),
    T.ToTensor(),
    # T.Lambda(lambda x: x.flatten()),
])

tsfms_increasing = T.Compose([
    T.RandomHorizontalFlip(),
    T.RandomResizedCrop(size=(150, 150), scale=(0.2, 0.8)),
    T.ToTensor(),
    T.Resize(size=(260, 260)),
    # T.Lambda(lambda x: x.flatten()),
])

train_dset = GroceryStoreDataset(
    split="train",
    transform=tsfms_std,
)
increased_train_dst = GroceryStoreDataset(
    split="train",
    transform=tsfms_increasing,
)
val_dset = GroceryStoreDataset(
    split="val",
    transform=tsfms_std,
)
test_dset = GroceryStoreDataset(
    split="test",
    transform=tsfms_std,
)
n_classes = 43
input_dim = len(train_dset[0][0])

increased_dataset = torch.utils.data.ConcatDataset([increased_train_dst,train_dset])
print(len(increased_dataset))

5280


In [57]:
from torch.utils.data import DataLoader


In [8]:
print(len(train_dset))
print(len(val_dset))

2640
296


In [ ]:
# _, ax = plt.subplots(1, 4)

# for i, idx in enumerate([0, 10, 20, 30]):
#     img, label = test_dset[idx]
#     # Obtain dimension for each axis (consider them already resized to 400x400)
#     dim = np.int32(np.ceil(np.sqrt(img.shape[0]/3)))

#     ax[i].imshow(img.reshape(3, dim, dim).permute(1, 2, 0))
#     ax[i].set_title(label)
#     ax[i].axis("off")

In [65]:
batch_size = 64

# first_train_loader = DataLoader(increased_dataset,
#                                 batch_size=batch_size,
#                                 shuffle=True)
# train_mean, train_std = get_mean_std(first_train_loader)

# first_val_loader = DataLoader(val_dset,
#                                 batch_size=batch_size,
#                             )
# val_mean, val_std = get_mean_std(first_val_loader)

# train_data_transforms = T.Compose([
#     T.Resize((260, 260)),
#     T.ToTensor(),
#     T.Normalize(mean=train_mean, std=train_std)
# ])

# val_data_transforms = T.Compose([
#     T.Resize((260, 260)),
#     T.ToTensor(),
#     T.Normalize(mean=val_mean, std=val_std)
# ])


train_dl = DataLoader(
    increased_dataset,
    batch_size=batch_size,
    shuffle=True
)
val_dl = DataLoader(
    val_dset,
    batch_size=batch_size
)
test_dl = DataLoader(
    test_dset,
    batch_size=batch_size
)

In [66]:
# next(iter(...)) catches the first batch of the data loader
# If shuffle is True, this will return a different batch every time we run this cell
# For iterating over the whole dataset, we can simple use "for batch in data_loader: ..."
data_inputs, data_labels = next(iter(train_dl))

# The shape of the outputs are [batch_size, d_1,...,d_N] where d_1,...,d_N are the
# dimensions of the data point returned from the dataset class
print(f"Data inputs: {data_inputs.shape}\n{data_inputs}")
print(f"\nData labels: {data_labels.shape}\n{data_labels}")

Data inputs: torch.Size([64, 3, 260, 260])
tensor([[[[0.7569, 0.7612, 0.7679,  ..., 0.0780, 0.0735, 0.0706],
          [0.7583, 0.7610, 0.7653,  ..., 0.0823, 0.0778, 0.0749],
          [0.7606, 0.7608, 0.7612,  ..., 0.0891, 0.0845, 0.0817],
          ...,
          [0.8683, 0.8683, 0.8683,  ..., 0.6601, 0.6535, 0.6494],
          [0.8457, 0.8457, 0.8457,  ..., 0.6400, 0.6360, 0.6336],
          [0.8314, 0.8314, 0.8314,  ..., 0.6272, 0.6250, 0.6235]],

         [[0.5922, 0.5965, 0.6032,  ..., 0.0704, 0.0681, 0.0667],
          [0.5922, 0.5954, 0.6005,  ..., 0.0705, 0.0699, 0.0695],
          [0.5922, 0.5938, 0.5963,  ..., 0.0708, 0.0728, 0.0741],
          ...,
          [0.7426, 0.7398, 0.7354,  ..., 0.5503, 0.5460, 0.5433],
          [0.7177, 0.7158, 0.7127,  ..., 0.5279, 0.5262, 0.5252],
          [0.7020, 0.7005, 0.6983,  ..., 0.5137, 0.5137, 0.5137]],

         [[0.0196, 0.0182, 0.0159,  ..., 0.0655, 0.0542, 0.0471],
          [0.0239, 0.0199, 0.0135,  ..., 0.0558, 0.0469, 0.0413],

## Prepare network

### complex Model 

In [ ]:
## Best Performance on Validation set: 0.23
class SimpleClassifier(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, n_classes, n_hidden_layers=0):
        super().__init__()

        # Initialize the modules we need to build the network
        # self.first = torch.nn.Linear(input_dim, hidden_dim)
        # self.activation = torch.nn.ReLU()
        # self.last = torch.nn.Linear(hidden_dim, n_classes)

        # self.hidden_layers = torch.nn.ModuleList([
        #     torch.nn.Linear(hidden_dim, hidden_dim) for i in range(n_hidden_layers)
        # ])
        self.relu = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax(dim=0)

        self.first = torch.nn.Conv2d(in_channels=input_dim, out_channels=16, kernel_size=5, stride=4, padding='valid')

        self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, padding='same')
        self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding='same')
        self.pool1 = torch.nn.MaxPool2d(2, 2)

        self.batchnorm1 = torch.nn.BatchNorm2d(num_features=32)

        self.conv4 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding='same')
        self.conv5 = torch.nn.Conv2d(in_channels=64, out_channels=64, kernel_size=5, padding='same')
        self.pool2 = torch.nn.MaxPool2d(2, 2)

        self.batchnorm2 = torch.nn.BatchNorm2d(num_features=64)

        self.conv6 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding='same')
        self.conv7 = torch.nn.Conv2d(in_channels=128, out_channels=128, kernel_size=5, padding='same')
        self.pool3 = torch.nn.MaxPool2d(2, 2)

        self.batchnorm3 = torch.nn.BatchNorm2d(num_features=128)

        self.conv8 = torch.nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, padding='same')
        self.conv9 = torch.nn.Conv2d(in_channels=256, out_channels=256, kernel_size=5, padding='same')
        self.pool4 = torch.nn.MaxPool2d(2, 2)

        self.batchnorm4 = torch.nn.BatchNorm2d(num_features=256)

        self.flatten1 = torch.nn.Flatten()
        # self.fc1 = torch.nn.Linear((hidden_dim*16) * 1 * 1, 120)
        self.fc1 = torch.nn.Linear(6400, 120)
        self.fc2 = torch.nn.Linear(120, n_classes)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.first(x)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.pool1(x)

        x = self.batchnorm1(x)

        x = self.conv4(x)
        x = self.relu(x)
        x = self.conv5(x)
        x = self.relu(x)
        x = self.pool2(x)

        x = self.batchnorm2(x)

        x = self.conv6(x)
        x = self.relu(x)
        x = self.conv7(x)
        x = self.relu(x)
        x = self.pool3(x)

        x = self.batchnorm3(x)

        x = self.conv8(x)
        x = self.relu(x)
        x = self.conv9(x)
        x = self.relu(x)
        x = self.pool4(x)

        x = self.batchnorm4(x)
        
        x = self.flatten1(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        x = self.softmax(x)

        return x

### Model6

In [67]:

class ResNet(torch.nn.Module):
    def __init__(self, module):
        super().__init__()
        self.module = module

    def forward(self, inputs):
        return self.module(inputs) + inputs

# Best Performance on Validation set: 0.5101351351351351 con step 3e-4
class SimpleClassifier6(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, n_classes, n_hidden_layers=0):
        super().__init__()

        dim = hidden_dim
        # self.conv_layer1 = torch.nn.Sequential(
        #     torch.nn.Conv2d(in_channels=input_dim, out_channels=dim, kernel_size=5, stride=2, padding='valid'),

        #     torch.nn.BatchNorm2d(num_features=dim),

        #     torch.nn.ReLU(),
        #     torch.nn.MaxPool2d(2, 2),

        # )

        # self.conv_layer2 = torch.nn.Sequential(
        #     torch.nn.Conv2d(in_channels=dim, out_channels=dim*2, kernel_size=3, padding='same'),

        #     torch.nn.BatchNorm2d(dim*2),

        #     torch.nn.ReLU(),
        #     torch.nn.MaxPool2d(2, 2),
        # )
        # dim = dim*2

        # self.conv_layer3 = torch.nn.Sequential(
        #     torch.nn.Conv2d(in_channels=dim, out_channels=dim*2, kernel_size=3, padding='same'),

        #     torch.nn.BatchNorm2d(dim*2),

        #     torch.nn.ReLU(),
        #     torch.nn.MaxPool2d(2, 2),

        #     # torch.nn.Dropout2d(p=0.2),
        # )
        # dim *= 2

        # self.conv_layer4 = torch.nn.Sequential(
        #     torch.nn.Conv2d(in_channels=dim, out_channels=dim*2, kernel_size=3, padding='same'),

        #     torch.nn.BatchNorm2d(dim*2),

        #     torch.nn.ReLU(),
        #     torch.nn.MaxPool2d(2, 2),
        # )
        # dim *= 2

        # self.conv_layer5 = torch.nn.Sequential(
        #     torch.nn.Conv2d(in_channels=dim, out_channels=dim*2, kernel_size=3, padding='same'),
        #     torch.nn.BatchNorm2d(dim*2),

        #     torch.nn.ReLU(),
        #     torch.nn.MaxPool2d(2, 2),
        # )
        # dim *= 2

        # Initialize the modules we need to build the network

        # self.fc_classifier = torch.nn.Sequential(
        #     torch.nn.Flatten(),
        #     torch.nn.Linear(28224, 100),
        #     torch.nn.ReLU(),
        #     # torch.nn.Linear(200, 100),
        #     # torch.nn.SiLU(),
        #     torch.nn.Dropout(p=0.5),
        #     torch.nn.Linear(100, n_classes),
        #     torch.nn.Softmax()
        # )
        self.silu = torch.nn.SiLU()
        self.dropout = torch.nn.Dropout(p=0.2)

        self.conv1 = torch.nn.Conv2d(in_channels=input_dim, out_channels=64, kernel_size=7, stride=4, padding='valid')
        self.pool1 = torch.nn.MaxPool2d(3, 2)
        self.conv2 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding='same')
        self.pool2 = torch.nn.MaxPool2d(3, 2)

        self.batchnorm1 = torch.nn.BatchNorm2d(num_features=128)

        self.residual_block1 = torch.nn.Sequential(
            ResNet(
                torch.nn.Sequential(
                    torch.nn.Conv2d(128, 128, kernel_size=3, padding='same'),
                    torch.nn.SiLU(),
                    torch.nn.BatchNorm2d(128),
                    torch.nn.Conv2d(128, 128, kernel_size=3, padding='same'),
                    # torch.nn.SiLU(),
                    torch.nn.BatchNorm2d(128),
                )
            ),
            torch.nn.SiLU()
        )
        

        self.conv3 = torch.nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding='same')
        self.pool3 = torch.nn.MaxPool2d(3, 2)

        self.batchnorm2 = torch.nn.BatchNorm2d(num_features=256)

        # self.residual_block2 = torch.nn.Sequential(
        #     ResNet(
        #         torch.nn.Sequential(
        #             torch.nn.Conv2d(256, 256, kernel_size=3, padding='same'),
        #             torch.nn.SiLU(),
        #             torch.nn.BatchNorm2d(256),
        #             torch.nn.Conv2d(256, 256, kernel_size=3, padding='same'),
        #             # torch.nn.SiLU(),
        #             torch.nn.BatchNorm2d(256),
        #         )
        #     ),
        #     torch.nn.SiLU()
        # )

        self.conv4 = torch.nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding='same')
        self.conv5 = torch.nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding='same')
        self.pool3 = torch.nn.MaxPool2d(3, 2)

        self.batchnorm3 = torch.nn.BatchNorm2d(num_features=512)
        
        # self.conv5 = torch.nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, padding='valid')
        # self.pool5 = torch.nn.MaxPool2d(2, 2)

        # self.batchnorm4 = torch.nn.BatchNorm2d(num_features=256)

        # self.conv6 = torch.nn.Conv2d(in_channels=256, out_channels=512, kernel_size=5, padding='valid')
        # self.pool6 = torch.nn.MaxPool2d(2, 2)

        self.adaptive_avg = torch.nn.AdaptiveAvgPool2d(output_size=(3, 3))

        # self.batchnorm5 = torch.nn.BatchNorm2d(num_features=512)

        self.flatten1 = torch.nn.Flatten()
        # self.fc1 = torch.nn.Linear(512 * 7 * 7, 120)
        self.fc1 = torch.nn.Linear(4608, 120)
        self.fc2 = torch.nn.Linear(120, n_classes)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.conv1(x)
        x = self.silu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.silu(x)
        x = self.pool2(x)

        x = self.batchnorm1(x)

        x = self.residual_block1(x)

        x = self.conv3(x)
        x = self.silu(x)
        x = self.pool3(x)

        x = self.batchnorm2(x)

        # x = self.residual_block2(x)

        x = self.conv4(x)
        x = self.silu(x)
        x = self.conv5(x)
        x = self.silu(x)
        x = self.pool3(x)

        x = self.batchnorm3(x)

        # x = self.conv5(x)
        # x = self.elu(x)
        # x = self.pool5(x)

        # x = self.batchnorm4(x)

        # x = self.conv6(x)
        # x = self.elu(x)
        # x = self.pool6(x)
        
        # x = self.batchnorm5(x)

        x = self.adaptive_avg(x)

        x = self.flatten1(x)
        x = self.fc1(x)
        x = self.silu(x)
        x = self.dropout(x)
        x = self.fc2(x)

        # x = self.conv_layer1(x)
        # x = self.conv_layer2(x)
        # x = self.conv_layer3(x)
        # x = self.conv_layer4(x)
        # x = self.conv_layer5(x)
        
        # x = self.fc_classifier(x)

        return x

### Prova

In [ ]:
class ComplexCNN(torch.nn.Module):
    def __init__(self, n_classes):
        super(ComplexCNN, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 64, kernel_size=5, stride=2, padding=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            torch.nn.Conv2d(64, 64, kernel_size=5, stride=2, padding=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(),
            torch.nn.Conv2d(128, 128, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 256, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(),
            torch.nn.Conv2d(256, 256, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(256, 512, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(512),
            torch.nn.ReLU(),
            torch.nn.Conv2d(512, 512, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(512),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.flatten = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(12800, 96)
        # self.fc1 = torch.nn.Linear(512*2*2, 96)
        self.dropout = torch.nn.Dropout(0.5)
        self.fc2 = torch.nn.Linear(96, n_classes)
        self.softmax = torch.nn.Softmax()

        self.relu = torch.nn.ReLU()
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

### Last Model

In [ ]:
# Best Performance on Validation set: 0.5101351351351351 con step 3e-4
class SimpleClassifier5(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, n_classes, n_hidden_layers=0):
        super().__init__()

        dim = 96
        self.conv_layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=input_dim, out_channels=dim, kernel_size=11, stride=4, padding='valid'),

            torch.nn.SiLU(),
            torch.nn.MaxPool2d(3, 2),

            torch.nn.BatchNorm2d(num_features=dim)

        )

        self.conv_layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=dim, out_channels=dim*2, kernel_size=3, stride=2, padding='valid'),

            torch.nn.SiLU(),
            torch.nn.MaxPool2d(2, 2),

            torch.nn.BatchNorm2d(dim*2)
        )
        dim = dim*2

        self.conv_layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=dim, out_channels=dim*2, kernel_size=5, padding='same'),

            torch.nn.SiLU(),
            torch.nn.MaxPool2d(2, 2),

            torch.nn.BatchNorm2d(dim*2),

            # torch.nn.Dropout2d(p=0.2),
        )
        dim *= 2

        self.conv_layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=dim, out_channels=dim*2, kernel_size=5, padding='same'),

            torch.nn.SiLU(),
            torch.nn.MaxPool2d(2, 2),

            torch.nn.BatchNorm2d(dim*2)
        )
        dim *= 2

        self.conv_layer5 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels=dim, out_channels=dim*2, kernel_size=5, padding='same'),

            torch.nn.SiLU(),
            torch.nn.MaxPool2d(2, 2),

            torch.nn.BatchNorm2d(dim*2)
        )
        dim *= 2

        # Initialize the modules we need to build the network

        self.fc_classifier = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(6400, 100),
            torch.nn.SiLU(),
            # torch.nn.Linear(200, 100),
            # torch.nn.SiLU(),
            torch.nn.Dropout(p=0.05),
            torch.nn.Linear(100, n_classes),
            torch.nn.LogSoftmax(dim=0)
        )
        # self.elu = torch.nn.ELU()
        # self.softmax = torch.nn.Softmax()

        # self.conv1 = torch.nn.Conv2d(in_channels=input_dim, out_channels=16, kernel_size=5, padding='valid')
        # self.pool1 = torch.nn.MaxPool2d(2, 2)
        # self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, padding='valid')
        # self.pool2 = torch.nn.MaxPool2d(2, 2)

        # self.batchnorm1 = torch.nn.BatchNorm2d(num_features=32)

        # self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, padding='valid')
        # self.pool3 = torch.nn.MaxPool2d(2, 2)
        # self.conv4 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding='valid')
        # self.pool4 = torch.nn.MaxPool2d(2, 2)

        # self.batchnorm2 = torch.nn.BatchNorm2d(num_features=128)

        # self.conv5 = torch.nn.Conv2d(in_channels=128, out_channels=256, kernel_size=5, padding='valid')
        # self.pool5 = torch.nn.MaxPool2d(2, 2)
        # self.conv6 = torch.nn.Conv2d(in_channels=256, out_channels=512, kernel_size=5, padding='valid')
        # self.pool6 = torch.nn.MaxPool2d(2, 2)

        # self.batchnorm3 = torch.nn.BatchNorm2d(num_features=512)
        
        

        # self.batchnorm4 = torch.nn.BatchNorm2d(num_features=64)

        

        # self.batchnorm5 = torch.nn.BatchNorm2d(num_features=32)

        # self.flatten1 = torch.nn.Flatten()
        # self.fc1 = torch.nn.Linear(1152, 120)
        # self.fc2 = torch.nn.Linear(120, n_classes)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        # x = self.conv1(x)
        # x = self.elu(x)
        # x = self.pool1(x)
        # x = self.conv2(x)
        # x = self.elu(x)
        # x = self.pool2(x)

        # x = self.batchnorm1(x)

        # x = self.conv3(x)
        # x = self.elu(x)
        # x = self.pool3(x)
        # x = self.conv4(x)
        # x = self.elu(x)
        # x = self.pool4(x)

        # x = self.batchnorm2(x)

        # x = self.conv5(x)
        # x = self.elu(x)
        # x = self.pool5(x)
        # x = self.conv6(x)
        # x = self.elu(x)
        # x = self.pool6(x)
        
        # x = self.batchnorm3(x)
        x = self.conv_layer1(x)
        x = self.conv_layer2(x)
        x = self.conv_layer3(x)
        x = self.conv_layer4(x)
        # x = self.conv_layer5(x)
        
        x = self.fc_classifier(x)

        return x

### Model1

Model1: prova (migliore) con 2 blocchi (Conv -> RELU -> MaxPool x2) intervallati da un BatchNorm layer.

In [ ]:
# Best Performance on Validation set: 0.310
class SimpleClassifier1(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, n_classes, n_hidden_layers=0):
        super().__init__()

        # Initialize the modules we need to build the network
        # self.first = torch.nn.Linear(input_dim, hidden_dim)
        # self.activation = torch.nn.ReLU()
        # self.last = torch.nn.Linear(hidden_dim, n_classes)

        # self.hidden_layers = torch.nn.ModuleList([
        #     torch.nn.Linear(hidden_dim, hidden_dim) for i in range(n_hidden_layers)
        # ])
        self.relu = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax()

        self.conv1 = torch.nn.Conv2d(in_channels=input_dim, out_channels=16, kernel_size=5)
        self.pool1 = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)
        self.pool2 = torch.nn.MaxPool2d(2, 2)


        self.batchnorm1 = torch.nn.BatchNorm2d(num_features=32)

        self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
        self.pool3 = torch.nn.MaxPool2d(2, 2)
        self.conv4 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5)
        self.pool4 = torch.nn.MaxPool2d(2, 2)

        self.batchnorm2 = torch.nn.BatchNorm2d(num_features=128)

        self.flatten1 = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(8192, 120)
        self.fc2 = torch.nn.Linear(120, n_classes)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool2(x)

        x = self.batchnorm1(x)

        x = self.conv3(x)
        x = self.relu(x)
        x = self.pool3(x)
        x = self.conv4(x)
        x = self.relu(x)
        x = self.pool4(x)

        x = self.batchnorm2(x)
        
        x = self.flatten1(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        x = self.softmax(x)

        return x

### Model2

Model2: prova con blocco Convolutional Layer -> RELU -> MaxPooling Layer, ripetuto due volte. Stride usato unicamente per il MaxPooling layers.

In [ ]:
# Best Performance on Validation set: 0.24225352112676057
class SimpleClassifier2(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, n_classes, n_hidden_layers=0):
        super().__init__()

        # Initialize the modules we need to build the network
        # self.first = torch.nn.Linear(input_dim, hidden_dim)
        # self.activation = torch.nn.ReLU()
        # self.last = torch.nn.Linear(hidden_dim, n_classes)

        # self.hidden_layers = torch.nn.ModuleList([
        #     torch.nn.Linear(hidden_dim, hidden_dim) for i in range(n_hidden_layers)
        # ])
        self.relu = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax()

        self.conv1 = torch.nn.Conv2d(in_channels=input_dim, out_channels=16, kernel_size=5)
        self.pool1 = torch.nn.MaxPool2d(2, 2)
        self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5)
        self.pool2 = torch.nn.MaxPool2d(2, 2)


        self.flatten1 = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(70688, 120)
        self.fc2 = torch.nn.Linear(120, n_classes)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool2(x)
        
        x = self.flatten1(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        x = self.softmax(x)

        return x

### Model3

Model3:
prova con un solo Convolutional Layer, a cui aumento kernel_size e stride per ridurne la dimensionalità e di conseguenza il numero di parametri. 

In [ ]:
# Best Performance on Validation set: 0.20321931589537223
class SimpleClassifier3(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, n_classes, n_hidden_layers=0):
        super().__init__()

        # Initialize the modules we need to build the network
        self.relu = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax()

        self.conv1 = torch.nn.Conv2d(in_channels=input_dim, out_channels=16, kernel_size=11, stride=6)


        self.flatten1 = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(16384, 120)
        self.fc2 = torch.nn.Linear(120, n_classes)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.conv1(x)
        x = self.relu(x)
        
        x = self.flatten1(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        x = self.softmax(x)

        return x

### Model4

Model4: prova con un convolutional Layer e un pooling Layer.

In [ ]:
# Best Performance on Validation set: 
class SimpleClassifier4(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, n_classes, n_hidden_layers=0):
        super().__init__()

        # Initialize the modules we need to build the network
        self.relu = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax()

        self.conv1 = torch.nn.Conv2d(in_channels=input_dim, out_channels=16, kernel_size=7, stride=4)
        self.pool1 = torch.nn.MaxPool2d(2, 2)

        self.flatten1 = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(9216, 120)
        self.fc2 = torch.nn.Linear(120, n_classes)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.conv1(x)
        x = self.relu(x)
        
        x = self.pool1(x)
        
        x = self.flatten1(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)

        x = self.softmax(x)

        return x

### Chiamata Modelli

In [68]:
# hidden_dim = 128

# model = SimpleClassifier(
#     3,
#     16,
#     n_classes,
# )

# model_best = SimpleClassifier1(
#     3,
#     16,
#     n_classes
# )

# model2 = SimpleClassifier2(
#     3,
#     16,
#     n_classes
# )

# model3 = SimpleClassifier3(
#     3,
#     16,
#     n_classes
# )

# model4 = SimpleClassifier4(
#     3,
#     16,
#     n_classes
# )

# model_complex = ComplexCNN(
#     n_classes
# )

# model5 = SimpleClassifier4(
#     3,
#     16,
#     n_classes
# )

model6 = SimpleClassifier6(
    3,
    32,
    n_classes
)

# model.forward(torch.zeros(size=(3, 200, 200)))

# model = SimpleClassifier(
#     input_dim,
#     hidden_dim,
#     n_classes,
#     n_hidden_layers=1
# )
for name, params in model6.named_parameters():
    print(f"{name}: {params.shape}")

conv1.weight: torch.Size([64, 3, 7, 7])
conv1.bias: torch.Size([64])
conv2.weight: torch.Size([128, 64, 5, 5])
conv2.bias: torch.Size([128])
batchnorm1.weight: torch.Size([128])
batchnorm1.bias: torch.Size([128])
residual_block1.0.module.0.weight: torch.Size([128, 128, 3, 3])
residual_block1.0.module.0.bias: torch.Size([128])
residual_block1.0.module.2.weight: torch.Size([128])
residual_block1.0.module.2.bias: torch.Size([128])
residual_block1.0.module.3.weight: torch.Size([128, 128, 3, 3])
residual_block1.0.module.3.bias: torch.Size([128])
residual_block1.0.module.4.weight: torch.Size([128])
residual_block1.0.module.4.bias: torch.Size([128])
conv3.weight: torch.Size([256, 128, 3, 3])
conv3.bias: torch.Size([256])
batchnorm2.weight: torch.Size([256])
batchnorm2.bias: torch.Size([256])
conv4.weight: torch.Size([512, 256, 3, 3])
conv4.bias: torch.Size([512])
conv5.weight: torch.Size([512, 512, 3, 3])
conv5.bias: torch.Size([512])
batchnorm3.weight: torch.Size([512])
batchnorm3.bias: torc

## Push the model to the device (CUDA) 

In [69]:
# Push model to device. Has to be done only once.
model6.to(device)

SimpleClassifier6(
  (silu): SiLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=valid)
  (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (batchnorm1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (residual_block1): Sequential(
    (0): ResNet(
      (module): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (1): SiLU()
        (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): SiLU()
  )
  (conv3): Conv2d(128, 

## Optimizer

In [15]:
from torch.optim import Adam

In [70]:
optimizer = Adam(model6.parameters(), lr=3e-5)

## Training the network

In [17]:

from tqdm.notebook import tqdm
import torch.nn.functional as F

In [18]:
def ncorrect(scores, y):
    y_hat = torch.argmax(scores, -1)
    return (y_hat == y).sum()

def accuracy(scores, y):
    correct = ncorrect(scores, y)
    return correct.true_divide(y.shape[0])

def train_loop(model, train_dl, epochs, opt, val_dl=None, verbose=False):
    best_val_acc = 0
    best_params = []
    best_epoch = -1

    for e in tqdm(range(epochs)):
    # for e in range(epochs):
        model.train()
        # Train
        train_loss = 0
        train_samples = 0
        train_acc = 0
        # running_loss = 0
        # correct = 0
        # total = 0
        for train_data in train_dl:
            imgs = train_data[0].to(device)
            labels = train_data[1].to(device)

            opt.zero_grad()  # clear

            # print(f'imgs = {imgs.shape}')
            scores = model(imgs)
            # print(f'scores= {scores.shape}')
            
            loss = F.cross_entropy(scores, labels)
            loss.backward()  # fill
            opt.step()       # use


            # running_loss += loss.item()
            # _, predicted = torch.max(scores.data, 1)
            # total += labels.size(0)
            # correct += (predicted == labels).sum().item()

            train_loss += loss.item() * imgs.shape[0]
            train_samples += imgs.shape[0]
            train_acc += ncorrect(scores, labels).item()

            
        # train_loss = running_loss / len(train_dl)
        # train_acc = 100 * correct / total
        train_acc /= train_samples
        train_loss /= train_samples

        # Validation
        model.eval()
        with torch.no_grad():
            val_loss = 0
            val_samples = 0
            val_acc = 0
            # correct = 0
            # total = 0
            if val_dl is not None:
                for val_data in val_dl:
                    imgs = val_data[0].to(device)
                    labels = val_data[1].to(device)
                    val_scores = model(imgs)
                    val_loss += F.cross_entropy(val_scores, labels).item() * imgs.shape[0]

                    val_samples += imgs.shape[0]
                    val_acc += ncorrect(val_scores, labels).item()

                    # val_scores = model(imgs)
                    # _, predicted = torch.max(val_scores.data, 1)
                    # total += labels.size(0)
                    # correct += (predicted == labels).sum().item()

                val_acc /= val_samples
                val_loss /= val_samples
                # val_acc = 100 * correct / total

            if val_dl is None or val_acc > best_val_acc:
                best_val_acc = val_acc if val_dl is not None else 0
                best_params = model.state_dict()
                torch.save(best_params, "best_model.pth")
                best_epoch = e

        # if verbose and e % 5 == 0:
        if verbose:
            print(f"Epoch {e}: train loss {train_loss:.3f} - train acc {train_acc:.3f}" + ("" if val_dl is None else f" - valid loss {val_loss:.3f} - valid acc {val_acc:.3f}"))
            # print(f"Epoch {e}: train loss {train_loss:.3f} - train acc {train_acc:.3f}" + ("" if val_dl is None else f" - valid loss - valid acc {val_acc:.3f}"))

    if verbose and val_dl is not None:
        print(f"Best epoch {best_epoch}, best acc {best_val_acc}")

    return best_val_acc, best_params, best_epoch

In [72]:
epochs = 30

best_val_acc, best_params, best_epoch = train_loop(
    model6,
    train_dl,
    epochs,
    optimizer,
    val_dl,
    verbose=True
)

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 0: train loss 2.684 - train acc 0.306 - valid loss 2.592 - valid acc 0.341
Epoch 1: train loss 1.789 - train acc 0.509 - valid loss 1.964 - valid acc 0.453
Epoch 2: train loss 1.303 - train acc 0.653 - valid loss 1.831 - valid acc 0.483
Epoch 3: train loss 1.014 - train acc 0.727 - valid loss 1.755 - valid acc 0.453
Epoch 4: train loss 0.817 - train acc 0.789 - valid loss 1.708 - valid acc 0.446
Epoch 5: train loss 0.687 - train acc 0.818 - valid loss 1.948 - valid acc 0.446
Epoch 6: train loss 0.582 - train acc 0.852 - valid loss 1.504 - valid acc 0.588
Epoch 7: train loss 0.512 - train acc 0.870 - valid loss 1.528 - valid acc 0.544
Epoch 8: train loss 0.447 - train acc 0.887 - valid loss 1.556 - valid acc 0.561
Epoch 9: train loss 0.428 - train acc 0.889 - valid loss 1.601 - valid acc 0.554
Epoch 10: train loss 0.374 - train acc 0.904 - valid loss 1.622 - valid acc 0.517
Epoch 11: train loss 0.353 - train acc 0.912 - valid loss 1.704 - valid acc 0.517
Epoch 12: train loss 0.302

In [71]:
torch.cuda.empty_cache()